Exports a basic hub/sattellite combo (using the SQL Server base) to a mapping sheet, suitable for import to the transforms tables.  
Things currently not done:  
 - adding the transform group name for the hash diff block  
 - adding the transform group logic for the hash diff block  
 - dealing with a multiple sattellite table - you need to just push the button 'twice' and handle it.  
 - complex hash keys    (this is a basic bitch script)

In [65]:
import json
import os
import sqlite3

from pathlib import Path
from datetime import datetime

conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "../export_output_files/TXT/"
output_filename = "databricks_vault_excel_" + time_string + ".txt"

In [66]:
def fetch_data_as_dict(cursor, query, params):
  try:
    cursor.execute(query, params)
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    data_dict_list = [dict(zip(columns, row)) for row in results]
    return data_dict_list
  except sqlite3.Error as e:
    print(f"Error: {e}")
    return []

In [67]:
# this should be the identifiers for the table you are drawing from
source_server_name = 'VHLOPRHP2S01\P2SLIVE'
source_database_name = 'Pro2_ih'
source_schema_name = 'dbo'
source_table_name = 're_tenancy'
key_column_name = 'tenancy_ref'
source_column_array = [
'prrowid'
, 'org_code'
, 'tenancy_ref'
, 'asoc_tncy'
, 'tncy_type'
, 'tncy_cat'
, 'rent_group'
, 'pmt_method'
, 'joint_single'
, 'private_council'
, 'stop_code'
, 'stop_code_end'
, 'tncy_start'
, 'tncy_end'
, 'corr_name##1'
, 'corr_name##2'
, 'curr_balance'
, 'prev_balance'
, 'decanted'
, 'tncy_status'
, 'net_rent'
, 'away_addr'
, 'former_addr'
, 'arrears_stage'
, 'prev_ars_stage'
, 'occupants'
, 'user_fields##1'
, 'user_fields##2'
, 'tncy_sys_ref'
, 'reinstated_tncy'
, 'term_reason'
, 'created_user'
, 'created_date'
, 'created_time'
, 'updated_user'
, 'updated_date'
, 'updated_time'
, 'tfr_reason'
, 'debt_counselling'
, 'arrears'
, 'prev_arrears'
, 'next_assured_inc'
, 'gross_rent'
, 'arrears_net_hb'
, 'debit'
, 'tresspasser'
, 'tresspasser_date'
, 'hold_pmt_sub_acc'
, 'profile_in_error'
, 'last_arrears_trans'
, 'core_rtn_sent'
, 'core_rtn_type'
, 'core_extracted_date'
, 'tenant_term_notify'
, 'arrears_net_sp'
, 'sp_arrears'
, 'hb_arrears'
, 'temp_acc'
, 'floating_support'
, 'oth_tncy_sys_refs'
, 'term_sub_reason'
, 'created_via'
, 'created_refno'
, 'last_payment_amt'
, 'first_inc_after_13feb2003'
, 'int_code'
, 'access_group'
, 'last_pay_account_code'
, 'last_pay_account_type'
, 'xxx_bank_sort_code'
, 'account_name'
, 'xxx_bank_acc_no'
, 'exp_tncy_end'
, 'barcode_ref'
, 'barcode_data'
, 'start_reason'
, 'start_sub_reason'
, 'bank_sortcode'
, 'bank_accountno'
, 'segment'
, 'manseg'
, 'landlord_acc'
, 'usage_code'
, 'Pro2SrcPDB'
, 'pro2created'
, 'pro2modified'
]
# this should be the identifiers for the vault that you are putting into
target_server_name = 'poc'
target_database_name = 'pocipc'
target_schema_name = 'rv'

target_entity_name = 'Tenancy'  # eg if the hub was h_Customer, 'Customer' would go here
target_hub_business_key_name = 'TenancyReference'  # the name of the field that the business key for the hub is going into eg 'CustomerReference'
target_sat_system_prefix = 'oh'

In [68]:
# this converts the identifiers into single strings for the queries
target_hub_name = 'h_' + target_entity_name
target_hub_hashkey_name = target_entity_name + 'HashKey'
target_hashkey_identifier = source_server_name + '.' + source_database_name + '.' + source_schema_name + '.' + source_table_name + '.' + key_column_name
target_sat_name = 's_' + target_entity_name + '_' + target_sat_system_prefix + '_' + source_table_name

In [69]:
# Get column list for the HashDiff field
query = '''
SELECT
    "''' + source_server_name + '''" AS SERVER_NAME_SOURCE
  , "''' + source_database_name + '''" AS DATABASE_NAME_SOURCE
  , "''' + source_schema_name + '''" AS SCHEMA_NAME_SOURCE
  , "''' + source_table_name + '''" AS TABLE_NAME_SOURCE
  , COLUMN_NAME AS COLUMN_NAME_SOURCE
  , "''' + target_server_name + '''" AS SERVER_NAME_TARGET
  , "''' + target_database_name + '''" AS DATABASE_NAME_TARGET
  , "''' + target_schema_name + '''" AS SCHEMA_NAME_TARGET
  , "''' + target_sat_name + '''" AS TABLE_NAME_TARGET
  , 'HashDiff' AS COLUMN_NAME_TARGET
  , 5 AS ORDINAL_POSITION
  , 'NO' AS IS_NULLABLE
  , 'STRING' AS DATA_TYPE
  , NULL AS TRANSFORM_LOGIC
  , NULL AS TRANSFORM_GROUP
  , NULL AS TRANSFORM_GROUP_SUFFIX
  , NULL AS TRANSFORM_GROUP_LOGIC
FROM bv_SQLServerPhysicalAttributeOutputForDataBricks
WHERE
  SERVER_NAME = ?
  AND TABLE_NAME = ?
  AND COLUMN_NAME IN ({})
'''.format(
  ",".join("?" for _ in source_column_array)
)
params = [source_server_name] + [source_table_name] + source_column_array
hash_dict_list = fetch_data_as_dict(cur, query, params)

#for row_dict in hash_dict_list:
#  print(row_dict)


In [70]:
# get field list for the rest of the satellie
query = '''
SELECT
    "''' + source_server_name + '''" AS SERVER_NAME_SOURCE
  , "''' + source_database_name + '''" AS DATABASE_NAME_SOURCE
  , "''' + source_schema_name + '''" AS SCHEMA_NAME_SOURCE
  , "''' + source_table_name + '''" AS TABLE_NAME_SOURCE
  , COLUMN_NAME AS COLUMN_NAME_SOURCE
  , "''' + target_server_name + '''" AS SERVER_NAME_TARGET
  , "''' + target_database_name + '''" AS DATABASE_NAME_TARGET
  , "''' + target_schema_name + '''" AS SCHEMA_NAME_TARGET
  , "''' + target_sat_name + '''" AS TABLE_NAME_TARGET
  , COLUMN_NAME AS COLUMN_NAME_TARGET
  , ROW_NUMBER() OVER (
    ORDER BY (SELECT NULL)
  ) + 5 AS ORDINAL_POSITION
  , IS_NULLABLE
  , DATA_TYPE
  , NULL AS TRANSFORM_LOGIC
  , NULL AS TRANSFORM_GROUP
  , NULL AS TRANSFORM_GROUP_SUFFIX
  , NULL AS TRANSFORM_GROUP_LOGIC
FROM bv_SQLServerPhysicalAttributeOutputForDataBricks
WHERE
  SERVER_NAME = ?
  AND TABLE_NAME = ?
  AND COLUMN_NAME IN ({})
'''.format(
  ",".join("?" for _ in source_column_array)
)
params = [source_server_name] + [source_table_name] + source_column_array
result_dict_list = fetch_data_as_dict(cur, query, params)

#for row_dict in result_dict_list:
#  print(row_dict)


In [71]:
# Set dict list of hub
hub_fields = [
  {
      'SERVER_NAME_SOURCE': source_server_name
    , 'DATABASE_NAME_SOURCE': source_database_name
    , 'SCHEMA_NAME_SOURCE': source_schema_name
    , 'TABLE_NAME_SOURCE': source_table_name
    , 'COLUMN_NAME_SOURCE': key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_hub_name
    , 'COLUMN_NAME_TARGET': target_hub_hashkey_name
    , 'ORDINAL_POSITION': 1
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'HASH(' + target_hashkey_identifier + ')'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_hub_name
    , 'COLUMN_NAME_TARGET': 'LoadDate'
    , 'ORDINAL_POSITION': 2
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'TIMESTAMP_NTZ'
    , 'TRANSFORM_LOGIC': 'DV load datetime'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_hub_name
    , 'COLUMN_NAME_TARGET': 'RecordSource'
    , 'ORDINAL_POSITION': 3
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': "''" + target_hashkey_identifier + "'"
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_hub_name
    , 'COLUMN_NAME_TARGET': 'ProcessID'
    , 'ORDINAL_POSITION': 4
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'System Process ID'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': source_server_name
    , 'DATABASE_NAME_SOURCE': source_database_name
    , 'SCHEMA_NAME_SOURCE': source_schema_name
    , 'TABLE_NAME_SOURCE': source_table_name
    , 'COLUMN_NAME_SOURCE': key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_hub_name
    , 'COLUMN_NAME_TARGET': target_hub_business_key_name
    , 'ORDINAL_POSITION': 5
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': None
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
]

# Set dict list of sat core fields
sat_fields = [
  {
      'SERVER_NAME_SOURCE': source_server_name
    , 'DATABASE_NAME_SOURCE': source_database_name
    , 'SCHEMA_NAME_SOURCE': source_schema_name
    , 'TABLE_NAME_SOURCE': source_table_name
    , 'COLUMN_NAME_SOURCE': key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_sat_name
    , 'COLUMN_NAME_TARGET': target_hub_hashkey_name
    , 'ORDINAL_POSITION': 1
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'HASH(' + target_hashkey_identifier + ')'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_sat_name
    , 'COLUMN_NAME_TARGET': 'LoadDate'
    , 'ORDINAL_POSITION': 2
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'TIMESTAMP_NTZ'
    , 'TRANSFORM_LOGIC': 'DV load datetime'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_sat_name
    , 'COLUMN_NAME_TARGET': 'RecordSource'
    , 'ORDINAL_POSITION': 3
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': "''" + source_server_name + '.' + source_database_name + '.' + source_schema_name + '.' + source_table_name + "'"
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_sat_name
    , 'COLUMN_NAME_TARGET': 'ProcessID'
    , 'ORDINAL_POSITION': 4
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'System Process ID'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
  }
]

# Append the lists
result_dict_list = hub_fields + sat_fields + hash_dict_list + result_dict_list


#for row_dict in result_dict_list:
#  print(row_dict)

In [72]:
# Write to a tab-separated text file
with open(os.path.join(output_path, output_filename), "w") as f:
  for row_dict in result_dict_list:
    line = "\t".join(str(row_dict[key]) for key in row_dict)
    f.write(line + "\n")

